<a href="https://colab.research.google.com/github/YianKim/2022_uncertainty_aware_semisupervise/blob/main/uncertaintylabeling_fer2013%2Bresnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

from tqdm import tqdm

## 데이터 불러오기


In [3]:
def load_data(dataset_path):
  
  #classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprsie', 'Neutral']

  data = []
  test_data = []
  test_labels = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            curr_class, line, set_type = line.split(',')
            image_data = np.asarray([int(x) for x in line.split()]).reshape(48, 48)
            image_data =image_data.astype(np.uint8)/255.0

            if (set_type.strip() == 'PrivateTest'):
              
              test_data.append(image_data)
              test_labels.append(curr_class)
            else:
              data.append(image_data)
              labels.append(curr_class)
      
      test_data = np.expand_dims(test_data, -1)
      test_labels = to_categorical(test_labels, num_classes = 7)
      data = np.expand_dims(data, -1)   
      labels = to_categorical(labels, num_classes = 7)
    
      return np.array(data), np.array(labels), np.array(test_data), np.array(test_labels)

dataset_path = "/content/drive/MyDrive/파이썬스터디 프로젝트/fer2013.csv/fer2013.csv"
train_data, train_labels, test_data, test_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

#평가용
test_labels2 = np.argmax(test_labels, axis=1)

# print("Number of images in Training set:", len(train_data))
# print("Number of images in Test set:", len(test_data))

with open('/content/drive/MyDrive/파이썬스터디 프로젝트/aug_array1.pkl', 'rb') as f:
	aug_array1 = pickle.load(f)
 
train_data_aug = np.concatenate((train_data, aug_array1), axis=0)
train_labels_aug = np.concatenate((train_labels, train_labels), axis=0)

In [4]:
train_data_aug.shape

(64596, 48, 48, 1)

In [5]:
train_labels_aug.shape

(64596, 7)

In [6]:
train_data = train_data_aug

In [7]:
train_labels = train_labels_aug

## resnet

In [17]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

def create_res_net():
    
    inputs = Input(shape=(48, 48, 1))
    num_filters = 32
    
    t = BatchNormalization()(inputs)
    # t = Conv2D(kernel_size=3,
    #            strides=1,
    #            filters=num_filters,
    #            padding="same")(t)
    # t = relu_bn(t)
    # t = PermaDropout(0.5)(t)
    
    num_blocks_list = [3,4,6,3]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
        t = PermaDropout(0.5)(t)
    
    t = AveragePooling2D(6)(t)
    t = Flatten()(t)
    t = PermaDropout(0.5)(t)
    outputs = Dense(7, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
model = create_res_net() # or create_plain_net()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 48, 48, 1)   4           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 48, 48, 32)   320         ['batch_normalization[0][0]']    
                                                                                                  
 re_lu (ReLU)                   (None, 48, 48, 32)   0           ['conv2d[0][0]']             

In [ ]:
# lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
# early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

# model.fit(
#     x=train_data,
#     y=train_labels,
#     epochs=500,
#     verbose=1,
#     validation_split=0.3,
#     batch_size=64,
#     callbacks=[lr_reducer, early_stopper]
# )

Epoch 1/500
270/270 [==============================] - 77s 234ms/step - loss: 1.9290 - accuracy: 0.2261 - val_loss: 2.1136 - val_accuracy: 0.2114 - lr: 0.0010
Epoch 2/500
270/270 [==============================] - 62s 229ms/step - loss: 1.7212 - accuracy: 0.3035 - val_loss: 1.6915 - val_accuracy: 0.3434 - lr: 0.0010
Epoch 3/500
270/270 [==============================] - 62s 229ms/step - loss: 1.5936 - accuracy: 0.3734 - val_loss: 1.5849 - val_accuracy: 0.3837 - lr: 0.0010
Epoch 4/500
270/270 [==============================] - 65s 240ms/step - loss: 1.4847 - accuracy: 0.4232 - val_loss: 1.5827 - val_accuracy: 0.4178 - lr: 0.0010
Epoch 5/500
270/270 [==============================] - 63s 232ms/step - loss: 1.3995 - accuracy: 0.4555 - val_loss: 1.4634 - val_accuracy: 0.4513 - lr: 0.0010
Epoch 6/500
270/270 [==============================] - 62s 229ms/step - loss: 1.3411 - accuracy: 0.4818 - val_loss: 1.4733 - val_accuracy: 0.4703 - lr: 0.0010
Epoch 7/500
270/270 [=========================

In [ ]:
# model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')

## 모델 결과

In [8]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [18]:
# model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')
model = create_res_net()
model.load_weights('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')
model_eval()

0.6912789077737531

# 라벨링

## 라벨링; 1

In [20]:
with open('/content/drive/MyDrive/파이썬스터디 프로젝트/face_array.pkl', 'rb') as f:
	unlab_data = pickle.load(f)

In [19]:
# model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')
model = create_res_net()
model.load_weights('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet.h5')

In [ ]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(20)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0], 'float32')
  
  for i in range(20):
    temp2 += temp1[i][j]
  Outs.append(temp2/20)
  
  outputs=[]

  for i in range(20):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

 30%|███       | 6/20 [07:57<18:25, 78.94s/it]

In [ ]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [ ]:
Counter(np.argmax(unlab_labels, axis=1))

### 불확정성


In [ ]:
# upper bound of variance?

UB25 = []
UB50 = []
UB75 = []


for h in range(7):
  classvars = []
  for i in tqdm(range(67224)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB25.append(np.percentile(classvars, 25))
  UB50.append(np.percentile(classvars, 50))
  UB75.append(np.percentile(classvars, 75))

In [ ]:
# UB25 < UB50 < UB75

In [ ]:
vars25 = []
vars50 = []
vars75 = []
vars100 = []

ind = 0 


for i in Vars:
  if i <= UB25[np.argmax(unlab_labels, axis=1)[ind]]:
    vars25.append(ind)
  elif i <= UB50[np.argmax(unlab_labels, axis=1)[ind]]:
    vars50.append(ind)
  elif i <= UB75[np.argmax(unlab_labels, axis=1)[ind]]:
    vars75.append(ind)
  else:
    vars100.append(ind)
  ind += 1

In [ ]:
k1 = random.sample(range(len(vars25)), len(vars25))
k2 = random.sample(range(len(vars50)), len(vars50))
k3 = random.sample(range(len(vars75)), len(vars75))
k4 = random.sample(range(len(vars100)), len(vars100))

lowvars = k1[0:len(k1)]+k2[0:len(k2)]
highvars = k3[0:len(k3)]+k4[0:len(k4)]

clstvars1 = k1[0:np.int(len(k1)/2)] + k2[0:np.int(len(k2)/2)] + k3[0:np.int(len(k3)/2)] + k4[0:np.int(len(k4)/2)]
clstvars2 = k1[np.int(len(k1)/2):len(k1)] + k2[np.int(len(k2)/2):len(k2)] + k3[np.int(len(k3)/2):len(k3)] + k4[np.int(len(k4)/2):len(k4)]

In [ ]:
train_data_1 = unlab_data[lowvars]
train_labels_1 = Outs[lowvars]

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = unlab_data[clstvars1]
train_labels_2 = Outs[clstvars1]

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
train_data_3 = unlab_data[randomindx]
train_labels_3 = Outs[randomindx]

In [ ]:
shufindx = random.sample(range(train_data_3.shape[0]),train_data_3.shape[0])
train_data_3 = train_data_3[shufindx]
train_labels_3 = train_labels_3[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[clstvars2]
unlab_labels_2 = unlab_labels[clstvars2]
unlab_data_3 = unlab_data[randomindx2]
unlab_labels_3 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

In [ ]:
Counter(np.argmax(train_labels_3, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

In [ ]:
Counter(np.argmax(unlab_labels_3, axis=1))

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

#### Resnet

In [ ]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def create_simple_net():
    
    inputs = Input(shape=(48, 48, 1))
    num_filters = 256
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = Dropout(0.5)(t)
    t = Flatten()(t)
    outputs = Dense(7, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_1,
    y=train_labels_1,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_a1.h5')

Epoch 1/500
219/219 [==============================] - 15s 56ms/step - loss: 8.2907 - accuracy: 0.3673 - val_loss: 4.2329 - val_accuracy: 0.2795 - lr: 0.0010
Epoch 2/500
219/219 [==============================] - 11s 48ms/step - loss: 5.4086 - accuracy: 0.5383 - val_loss: 4.9902 - val_accuracy: 0.2181 - lr: 0.0010
Epoch 3/500
219/219 [==============================] - 10s 47ms/step - loss: 3.5815 - accuracy: 0.6332 - val_loss: 2.6574 - val_accuracy: 0.5352 - lr: 0.0010
Epoch 4/500
219/219 [==============================] - 10s 47ms/step - loss: 2.5584 - accuracy: 0.7035 - val_loss: 2.6286 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 5/500
219/219 [==============================] - 10s 47ms/step - loss: 1.8950 - accuracy: 0.7544 - val_loss: 2.1456 - val_accuracy: 0.6914 - lr: 0.0010
Epoch 6/500
219/219 [==============================] - 10s 47ms/step - loss: 1.4230 - accuracy: 0.7897 - val_loss: 1.6314 - val_accuracy: 0.7292 - lr: 0.0010
Epoch 7/500
219/219 [==============================]

In [ ]:
model_eval()

0.5207578712733352

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_2,
    y=train_labels_2,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_b1.h5')

Epoch 1/500
219/219 [==============================] - 11s 49ms/step - loss: 8.1805 - accuracy: 0.3767 - val_loss: 12.1101 - val_accuracy: 0.0095 - lr: 0.0010
Epoch 2/500
219/219 [==============================] - 10s 46ms/step - loss: 5.2286 - accuracy: 0.5603 - val_loss: 3.2864 - val_accuracy: 0.2985 - lr: 0.0010
Epoch 3/500
219/219 [==============================] - 10s 47ms/step - loss: 3.5063 - accuracy: 0.6630 - val_loss: 2.0741 - val_accuracy: 0.6173 - lr: 0.0010
Epoch 4/500
219/219 [==============================] - 10s 46ms/step - loss: 2.6133 - accuracy: 0.7116 - val_loss: 2.5500 - val_accuracy: 0.6628 - lr: 0.0010
Epoch 5/500
219/219 [==============================] - 10s 45ms/step - loss: 1.9451 - accuracy: 0.7569 - val_loss: 2.0711 - val_accuracy: 0.7178 - lr: 0.0010
Epoch 6/500
219/219 [==============================] - 10s 46ms/step - loss: 1.4978 - accuracy: 0.7910 - val_loss: 1.6800 - val_accuracy: 0.7432 - lr: 0.0010
Epoch 7/500
219/219 [==============================

KeyboardInterrupt: ignored

In [ ]:
model_eval()

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
model = create_simple_net()

model.fit(
    x=train_data_3,
    y=train_labels_3,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

for i in range(len(model.layers)-1):
    model.layers[i].trainable = False
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')

model.fit(
    x=train_data,
    y=train_labels,
    epochs=500,
    verbose=1,
    validation_split=0.3,
    batch_size=64,
    callbacks=[lr_reducer, early_stopper]
)

model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_fer_resnet_c1.h5')

In [ ]:
model_eval()